In [1]:

import warnings
import json
import torchvision
import numpy as np
import time
import datetime
import os
import warnings
import pandas as pd
from PIL import Image
from glob import glob
from tqdm.auto import tqdm
import warnings

import torchvision

import matplotlib.pyplot as plt
# 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore")


In [2]:
import numpy as np
import time
import datetime
import os
import warnings
import pandas as pd
from PIL import Image
from glob import glob
from tqdm.auto import tqdm
import warnings

import torchvision

import matplotlib.pyplot as plt
# 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore")


def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")


def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result


def Preprocessing(file_list,dataset_calss, label_data):
    start = time.time()
    d = datetime.datetime.now()
    now_time = f"{d.year}-{d.month}-{d.day} {d.hour}:{d.minute}:{d.second}"
    print(f'[Preprocessing Start]')
    print(f'Preprocessing Start Time : {now_time}')
    frame_path = '../../data/valum/'+dataset_calss+'/'    
    for i in tqdm(range(len(file_list))):
       
        count = 0
        vidcap = torchvision.io.read_video(file_list[i])
        fps = int(vidcap[2]['video_fps'])
        video = np.array(vidcap[0], dtype=np.uint8)
        video_crop = np.zeros(
            (len(video)-1, video.shape[1], video.shape[2], 3))
        for j in range(len(video_crop)):
            video_crop[j] = video[j+1]-video[j]
        video_crop = video_crop.sum(axis=0)
        video_crop = video_crop.sum(axis=2)
        video_crop = ((video_crop/video_crop.max())*255).astype(np.uint8)
        y1 = np.where(video_crop > 200)[0].min()
        y2 = np.where(video_crop > 200)[0].max()
        x1 = np.where(video_crop > 200)[1].min()
        x2 = np.where(video_crop > 200)[1].max()
        video_name = os.path.basename(file_list[i])
        dst_label = label_data.loc[label_data["File Name"] == video_name]
        wake = str(dst_label['구분값'].item())
        Serial_Number = str(dst_label['일련번호'].item())
        file_name = Serial_Number+wake
        createDirectory(frame_path+file_name)
        for k in range(0, len(video), fps//5):
            img = Image.fromarray(video[k, y1:y2, x1:x2])
            img.resize((256, 256)).save(
                frame_path+file_name+"/%06d.jpg" % count)
            count += 1
    
    end = time.time()
    d = datetime.datetime.now()
    now_time = f"{d.year}-{d.month}-{d.day} {d.hour}:{d.minute}:{d.second}"
    print(f'Preprocessing Time : {now_time}s Time taken : {end-start}')
    print(f'[Preprocessing End]')

In [4]:
label_data = pd.DataFrame(columns=['구분값', '일련번호', 'velum','File Name'])
train_label_data = pd.DataFrame(columns=['구분값', '일련번호', 'velum','File Name'])
test_label_data = pd.DataFrame(columns=['구분값', '일련번호', 'velum','File Name'])
val_label_data = pd.DataFrame(columns=['구분값', '일련번호', 'velum','File Name'])
classes = ['no', 'partial', 'complete']
label_list=glob('../../../../YS_Baik/5.NIA_42/02_2_classification_gachon/Velum/01_data/train/*.json')
train_count=0
test_count=0
val_count=0
for i in tqdm(range(len(label_list))):
    with open(label_list[i], 'r') as f:
        json_data = json.load(f)
    f.close()

    try:
        separation_value= json_data['videos']['id'][json_data['videos']['id'].find('_')-1]
        serial_number=json_data['videos']['id'][:json_data['videos']['id'].find('_')-1]
        causes_of_velum=json_data['videos']['id']
        label_data.loc[train_count]=[int(separation_value),int(serial_number),classes.index(json_data['metas']['obstruction'])+1,json_data['videos']['filename']]
        train_label_data.loc[train_count]=[int(separation_value),int(serial_number),classes.index(json_data['metas']['obstruction'])+1,json_data['videos']['filename']]
        train_count+=1
    except:
        print(json_data['videos']['id'])
train_label_data.to_csv('../../data/valum/train_label.csv',index=False)
label_list=glob('../../../../YS_Baik/5.NIA_42/02_2_classification_gachon/Velum/01_data/test/*.json')
for i in tqdm(range(len(label_list))):
    with open(label_list[i], 'r') as f:
        json_data = json.load(f)
    f.close()

    try:
        separation_value= json_data['videos']['id'][json_data['videos']['id'].find('_')-1]
        serial_number=json_data['videos']['id'][:json_data['videos']['id'].find('_')-1]
        causes_of_velum=json_data['videos']['id']
        label_data.loc[train_count+test_count]=[int(separation_value),int(serial_number),classes.index(json_data['metas']['obstruction'])+1,json_data['videos']['filename']]
        test_label_data.loc[test_count]=[int(separation_value),int(serial_number),classes.index(json_data['metas']['obstruction'])+1,json_data['videos']['filename']]
        test_count+=1
    except:
        print(json_data['videos']['id'])
test_label_data.to_csv('../../data/valum/test_label.csv',index=False)
label_list=glob('../../../../YS_Baik/5.NIA_42/02_2_classification_gachon/Velum/01_data/val/*.json')
for i in tqdm(range(len(label_list))):
    with open(label_list[i], 'r') as f:
        json_data = json.load(f)
    f.close()
   
    try:
        separation_value= json_data['videos']['id'][json_data['videos']['id'].find('_')-1]
        serial_number=json_data['videos']['id'][:json_data['videos']['id'].find('_')-1]
        causes_of_velum=json_data['videos']['id']
        label_data.loc[train_count+test_count+val_count]=[int(separation_value),int(serial_number),classes.index(json_data['metas']['obstruction'])+1,json_data['videos']['filename']]
        val_label_data.loc[val_count]=[int(separation_value),int(serial_number),classes.index(json_data['metas']['obstruction'])+1,json_data['videos']['filename']]
        val_count+=1
    except:
        print(json_data['videos']['id'])
val_label_data.to_csv('../../data/valum/val_label.csv',index=False)
label_data.to_csv('../../data/valum/label.csv',index=False)


  0%|          | 0/3761 [00:00<?, ?it/s]

  0%|          | 0/471 [00:00<?, ?it/s]

  0%|          | 0/471 [00:00<?, ?it/s]

In [5]:
label_data = pd.read_csv('../../data/valum/label.csv')
train_label_data = pd.read_csv('../../data/valum/train_label.csv')
test_label_data = pd.read_csv('../../data/valum/test_label.csv')
val_label_data = pd.read_csv('../../data/valum/val_label.csv')



In [7]:
file_list = []
for i in range(len(train_label_data)):
    file_list.append('../../../../YS_Baik/5.NIA_42/02_2_classification_gachon/Velum/01_data/train/'+train_label_data['File Name'][i])
Preprocessing(file_list,'train', train_label_data)
file_list = []
for i in range(len(test_label_data)):
    file_list.append('../../../../YS_Baik/5.NIA_42/02_2_classification_gachon/Velum/01_data/test/'+test_label_data['File Name'][i])
Preprocessing(file_list,'test', test_label_data)
file_list = []
for i in range(len(val_label_data)):
    file_list.append('../../../../YS_Baik/5.NIA_42/02_2_classification_gachon/Velum/01_data/val/'+val_label_data['File Name'][i])
Preprocessing(file_list,'val', val_label_data)

[Preprocessing Start]
Preprocessing Start Time : 2024-6-3 14:16:31


  0%|          | 0/3761 [00:00<?, ?it/s]

Exception ignored in: <function tqdm.__del__ at 0x7f7b6af260d0>
Traceback (most recent call last):
  File "/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/tqdm/std.py", line 1149, in __del__
    self.close()
  File "/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/tqdm/notebook.py", line 281, in close
    self.disp(bar_style='success', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


Preprocessing Time : 2024-6-3 18:8:17s Time taken : 13905.776525259018
[Preprocessing End]
[Preprocessing Start]
Preprocessing Start Time : 2024-6-3 18:8:17


  0%|          | 0/471 [00:00<?, ?it/s]

Preprocessing Time : 2024-6-3 18:41:11s Time taken : 1973.983473777771
[Preprocessing End]
[Preprocessing Start]
Preprocessing Start Time : 2024-6-3 18:41:11


  0%|          | 0/471 [00:00<?, ?it/s]

Preprocessing Time : 2024-6-3 19:33:45s Time taken : 3153.7880914211273
[Preprocessing End]
